In [39]:
import ROOT
from ROOT import gROOT, TString
from ROOT import TCanvas, TLine, TGaxis, TGraph, gPad, TF1, TF2, kRed, TMultiGraph, TLegend, gStyle, TPaveStats, TStyle, TText, TList, TLatex, TGraphErrors, TFile, TTree
import json
import math
from array import array
import numpy as np
import pandas as pd

In [40]:
sigma_factor = (6.2*10e-3)/1.5

def HVCurrentDataFrame(scanIds):
    current_top = []    
    current_bot = []    
    HV_top = []    
    HV_bot = []
    eff = []
    eff_error = []
    muon_stream = []
    gamma_stream = []
    muon_CS = []
    gamma_CS = []
    muon_CM = []
    gamma_CM = []
    muon_CS_err = []
    gamma_CS_err = []
    muon_CM_err = []
    gamma_CM_err = []
    noiseGammaRate = []
    deltaV_top = []
    deltaV_bot = []
    deltaV_top_err = []
    deltaV_bot_err = []
    for scanId in scanIds:

        
        !echo "None" >> counter.txt
        !rm counter.txt
        path = "Scans_2024/Scan_00"+str(scanId)+"/*_HV*_CAEN.root"
        !ls {path} >> counter.txt
        counter = np.loadtxt("counter.txt", dtype=str)
        N = len(counter)+1

        
        for i in range(1,N):
            jsonFile = open("Scans_2024/Scan_00"+str(scanId)+"/ANALYSIS/KODELE/HV"+str(i)+"/output.json")
            jsonFile = json.load(jsonFile)
            histFile  = ROOT.TFile.Open("Scans_2024/Scan_00"+str(scanId)+"/Scan00"+str(scanId)+"_HV"+str(i)+"_CAEN.root","READ")
            hist_current_bot = histFile.Get("Imon_KODELE-BOT")
            hist_current_top = histFile.Get("Imon_KODELE-TOP")
            hist_HV_bot = histFile.Get("HVeff_KODELE-BOT")
            hist_HV_top = histFile.Get("HVeff_KODELE-TOP")
            eff.append(jsonFile['output_parameters']['efficiencyMuon_corrected'])
            eff_error.append(jsonFile['output_parameters']['efficiencyMuon_corrected_err'])
            muon_stream.append(jsonFile['output_parameters']['muonStreamerProbability'])
            gamma_stream.append(jsonFile['output_parameters']['gammaStreamerProbability'])
            muon_CS.append(jsonFile['output_parameters']['muonCLS'])
            gamma_CS.append(jsonFile['output_parameters']['gammaCLS'])
            muon_CM.append(jsonFile['output_parameters']['muonCMP'])
            gamma_CM.append(jsonFile['output_parameters']['gammaCMP'])
            muon_CS_err.append(jsonFile['output_parameters']['muonCLS_err'])
            gamma_CS_err.append(jsonFile['output_parameters']['gammaCLS_err'])
            muon_CM_err.append(jsonFile['output_parameters']['muonCMP_err'])
            gamma_CM_err.append(jsonFile['output_parameters']['gammaCMP_err'])
            noiseGammaRate.append(jsonFile['output_parameters']['noiseGammaRate'])
            current_bot.append(hist_current_bot.GetMean())
            current_top.append(hist_current_top.GetMean())
            HV_top.append(hist_HV_top.GetMean())
            HV_bot.append(hist_HV_bot.GetMean())
            deltaV_top.append(hist_current_top.GetMean()*1.5)
            deltaV_bot.append(hist_current_bot.GetMean()*1.5)
            deltaV_top_err.append(hist_current_top.GetMean()*6.2*10e-3)
            deltaV_bot_err.append(hist_current_bot.GetMean()*6.2*10e-3)
            histFile.Close()

    
    DataSet = {'HV_top': HV_top, 
               'HV_bot': HV_bot, 
               'current_top': current_top, 
               'current_bot': current_bot,
               'muon_stream': muon_stream,
               'gamma_stream': gamma_stream,
               'muon_CM': muon_CM,
               'gamma_CM': gamma_CM,
               'muon_CS': muon_CS,
               'gamma_CS': gamma_CS,
               'muon_CM_err': muon_CM_err,
               'gamma_CM_err': gamma_CM_err,
               'muon_CS_err': muon_CS_err,
               'gamma_CS_err': gamma_CS_err,
               'efficiency': eff,
               'eff_error': eff_error,
               'noiseGammaRate': noiseGammaRate,
               'deltaV': (np.array(deltaV_top)*np.array(deltaV_bot)+np.full_like(np.array(deltaV_bot), 0.00000001))/(np.array(deltaV_top)+np.array(deltaV_bot)+np.full_like(np.array(deltaV_bot), 0.00000001)),
               }
    
    DataSet = pd.DataFrame(data=DataSet)
    DataSet['current'] = DataSet['current_top'] + DataSet['current_bot']
    DataSet['deltaV_err'] = DataSet['deltaV']* sigma_factor *np.sqrt( (1 + (np.sqrt(DataSet['current_top']**2 + DataSet['current_bot']**2 ) / ( DataSet['current_top']+DataSet['current_bot']  ) )**2 ) ) 
    return DataSet

In [41]:
def FeaturesDataFrame(scanIds):
    
    current_top = []    
    current_bot = []    
    eff = []
    eff_error = []
    muon_stream = []
    gamma_stream = []
    muon_CS = []
    gamma_CS = []
    muon_CM = []
    gamma_CM = []
    muon_CS_err = []
    gamma_CS_err = []
    muon_CM_err = []
    gamma_CM_err = []
    current_bot = []
    current_top = []
    HV_top = []
    HV_bot = []
    noiseGammaRate = []
    noiseGammaRate_err = []

    
    for scanId in scanIds:
        print("scanID: ", scanIds[scanId][0])
        N = "1"
        
        jsonFile = open("Scans_2024/Scan_00"+str(scanIds[scanId][0])+"/ANALYSIS/KODELE/HV"+str(N)+"/output.json")
        histFile  = ROOT.TFile.Open("Scans_2024/Scan_00"+str(scanIds[scanId][0])+"/Scan00"+str(scanIds[scanId][0])+"_HV"+str(N)+"_CAEN.root","READ")
        hist_current_bot = histFile.Get("Imon_KODELE-BOT")
        hist_current_top = histFile.Get("Imon_KODELE-TOP")
        hist_HV_bot = histFile.Get("HVeff_KODELE-BOT")
        hist_HV_top = histFile.Get("HVeff_KODELE-TOP")
        jsonFile = json.load(jsonFile)
        nTrig = jsonFile['output_parameters']['nTrig']
        eff.append(100*jsonFile['output_parameters']['efficiencyMuon_corrected'])
        eff_error.append(jsonFile['output_parameters']['efficiencyMuon_corrected_err'])
        muon_stream.append(jsonFile['output_parameters']['muonStreamerProbability'])
        gamma_stream.append(jsonFile['output_parameters']['gammaStreamerProbability'])
        muon_CS.append(jsonFile['output_parameters']['muonCLS'])
        gamma_CS.append(jsonFile['output_parameters']['gammaCLS'])
        muon_CM.append(jsonFile['output_parameters']['muonCMP'])
        gamma_CM.append(jsonFile['output_parameters']['gammaCMP'])
        muon_CS_err.append(jsonFile['output_parameters']['muonCLS_err'])
        gamma_CS_err.append(jsonFile['output_parameters']['gammaCLS_err'])
        muon_CM_err.append(jsonFile['output_parameters']['muonCMP_err'])
        gamma_CM_err.append(jsonFile['output_parameters']['gammaCMP_err'])
        noiseGammaRate.append(jsonFile['output_parameters']['noiseGammaRate'])
        noiseGammaRate_err.append(jsonFile['output_parameters']['noiseGammaRate_err'])
        current_bot.append(hist_current_bot.GetMean())
        current_top.append(hist_current_top.GetMean())
        HV_top.append(hist_HV_top.GetMean())
        HV_bot.append(hist_HV_bot.GetMean())
        
    DataSet = {'current_top': current_top, 
               'current_bot': current_bot,
               'HV_top': HV_top,
               'HV_bot': HV_bot,
               'muon_stream': muon_stream,
               'gamma_stream': gamma_stream,
               'muon_CM': muon_CM,
               'gamma_CM': gamma_CM,
               'muon_CS': muon_CS,
               'gamma_CS': gamma_CS,
               'muon_CM_err': muon_CM_err,
               'gamma_CM_err': gamma_CM_err,
               'muon_CS_err': muon_CS_err,
               'gamma_CS_err': gamma_CS_err,
               'efficiency': eff,
               'eff_error': eff_error,
               'noiseGammaRate': noiseGammaRate,
               'noiseGammaRate_err': noiseGammaRate_err
              }
        
    DataSet = pd.DataFrame(data=DataSet)
    DataSet['current'] = DataSet['current_top'] + DataSet['current_bot']
    
    return DataSet

In [42]:
# Dicionários de Scans
scans = {
    'STDMX_OFF': ['6001'],
    'STDMX_22':  ['6009'],
    'STDMX_10':  ['6011'],
    'STDMX_6.9': ['6007'],
    'STDMX_3.3': ['6014'],
    'STDMX_2.2': ['5999'],
    'STDMX_1':   ['6005'],
}

#Scans no WP
Scans_STDMX = {
    'STDMX_OFF':  ['6004'],
    'STDMX_22':   ['6010'],
    'STDMX_10':   ['6012'],
    'STDMX_6.9':  ['6008'],
    'STDMX_3.3':  ['6003'],
    'STDMX_2.2':  ['6000'],
    'STDMX_1':    ['6006']
}

In [43]:
for scan in scans:
    print(scan, ': ')
    print(HVCurrentDataFrame(scans[scan]))

STDMX_OFF : 
   HV_top  HV_bot  current_top  current_bot  muon_stream  gamma_stream  \
0  6300.0  6300.0     2.968333     0.200000     0.000959      0.000000   
1  6500.0  6500.0     3.472696     0.200000     0.001931      0.000000   
2  6700.0  6700.0     3.953333     0.200000     0.005479      0.000000   
3  6900.0  6900.0     4.851667     0.400000     0.012658      0.000000   
4  7100.0  7100.0     5.250000     0.521667     0.022967      0.000544   
5  7200.0  7200.0     5.528333     0.611667     0.015640      0.000273   
6  7300.0  7300.0     5.916667     1.046667     0.032015      0.000000   
7  7400.0  7400.0     6.770000     1.170000     0.036310      0.001231   
8  7500.0  7500.0     7.435000     1.315000     0.048372      0.000549   
9  7600.0  7600.0     8.281667     1.543333     0.073171      0.000827   

    muon_CM  gamma_CM   muon_CS  gamma_CS  muon_CM_err  gamma_CM_err  \
0  1.000000  1.413043  2.777778  1.307692     0.111111      0.000000   
1  1.000000  1.303030  1.500

In [44]:
for scan in scans:
    df = HVCurrentDataFrame(scans[scan])  # Convertendo os dados para um DataFrame
    filename = f"{scan}.csv"  # Nome do arquivo baseado na chave do dicionário
    df.to_csv(filename, index=False)  # Salvando sem o índice
    print(f"Arquivo salvo: {filename}")

Arquivo salvo: STDMX_OFF.csv
Arquivo salvo: STDMX_22.csv
Arquivo salvo: STDMX_10.csv
Arquivo salvo: STDMX_6.9.csv
Arquivo salvo: STDMX_3.3.csv
Arquivo salvo: STDMX_2.2.csv
Arquivo salvo: STDMX_1.csv


In [45]:
import pandas as pd

for scan in Scans_STDMX:
    df = FeaturesDataFrame({scan: Scans_STDMX[scan]})  
    filename = f"{scan}_WP.csv"  
    df.to_csv(filename, index=False)  
    print(f"Arquivo salvo: {filename}")

scanID:  6004
Arquivo salvo: STDMX_OFF_WP.csv
scanID:  6010
Arquivo salvo: STDMX_22_WP.csv
scanID:  6012
Arquivo salvo: STDMX_10_WP.csv
scanID:  6008
Arquivo salvo: STDMX_6.9_WP.csv
scanID:  6003
Arquivo salvo: STDMX_3.3_WP.csv
scanID:  6000
Arquivo salvo: STDMX_2.2_WP.csv
scanID:  6006
Arquivo salvo: STDMX_1_WP.csv


In [ ]:
import ROOT
import pandas as pd
import os
import math
from ROOT import TLine, TLegend

num_files = int(input("Quantos scans deseja analisar? "))

csv_files = []
csv_WP_files = []
HV_ref = 95

for i in range(num_files):
    file_name = input(f"Digite o nome do arquivo {i+1} (ex: STDMX_1.csv): ")
    csv_files.append(file_name)

    base_name, ext = os.path.splitext(file_name)
    csv_WP_files.append(f"{base_name}_WP{ext}")

print("Arquivos WP correspondentes:", csv_WP_files)

# Canvas e gráficos
c1 = ROOT.TCanvas("c1", "Efficiency vs HV_top with Fits", 630, 600)
mg = ROOT.TMultiGraph()

colors = [ROOT.kBlue, ROOT.kRed, ROOT.kGreen+2, ROOT.kMagenta, ROOT.kOrange, ROOT.kCyan, ROOT.kBlack]
markers = [20, 21, 22, 23, 24, 25, 26]

graphs = []
fits = []

for i, file in enumerate(csv_files):
    df = pd.read_csv(file)
    n = len(df)
    gr = ROOT.TGraphErrors(n, 
                           df['HV_top'].values.astype(float), 
                           df['efficiency'].values.astype(float),
                           ROOT.nullptr,
                           df['eff_error'].values.astype(float))
    
    color = colors[i % len(colors)]
    marker = markers[i % len(markers)]
    
    gr.SetMarkerStyle(marker)
    gr.SetMarkerColor(color)
    gr.SetLineColor(color)
    gr.SetTitle(file.replace(".csv", ""))
    
    mg.Add(gr)
    graphs.append(gr)

    # Ajuste sigmoid
    sigmoid = ROOT.TF1(f"sigmoid_{i}", "[0]/(1+ TMath::Exp(-[1]*(x-[2])))", df['HV_top'].min(), df['HV_top'].max())
    sigmoid.SetParNames("Emax", "Lambda", "HV50")
    sigmoid.SetParameters(0.9, 0.01, 7000)
    sigmoid.SetLineColor(color)
    gr.Fit(sigmoid, "R")
    fits.append(sigmoid)

Emax = []
Emax_err = []
Lambda = []
Lambda_err = []
HV50 = []
HV50_err = []
HV95 = []
HV95_err = []
WP = []

for sigmoid in fits:
    Emax_ = 1 - math.sqrt((1 - sigmoid.GetParameter(0))**2)
    Emax.append(Emax_)
    
    Emax_err.append(sigmoid.GetParError(0))     
    Lambda.append(sigmoid.GetParameter(1))  
    Lambda_err.append(sigmoid.GetParError(1))
    HV50.append(sigmoid.GetParameter(2))  
    HV50_err.append(sigmoid.GetParError(2)) 
    HV95_val = sigmoid.GetX(HV_ref)
    HV95.append(HV95_val)
    HV95_err.append((math.log(19) / sigmoid.GetParameter(1)**2) * sigmoid.GetParError(1) + sigmoid.GetParError(2))  # erro HV95

for Emax_, Lambda_, HV50_ in zip(Emax, Lambda, HV50):
    WP.append(HV50_ - math.log(1 / 0.95 - 1) / Lambda_ + 150.)

legend = TLegend(0.12, 0.68, 0.3, 0.89)
legend.SetTextFont(42)
legend.SetBorderSize(0) 
legend.SetFillStyle(4000)
legend.SetFillColor(0)  
legend.SetTextSize(0.02376)  

for i, file in enumerate(csv_WP_files):
    df = pd.read_csv(file)
    if i == 0:
        eff_i_STDMX = f'plateau = {Emax[i]:.0%}, WP = {(WP[i]/1000):.2f} kV, no hit bkg gamma rate, Eff(WP) = {fits[i].Eval(WP[i]):.0%}'
        legend.AddEntry(graphs[i], eff_i_STDMX, "p")
    else:
        txt = df['noiseGammaRate'][0] / (df['gamma_CS'][0] * 1000)
        eff_i_STDMX = f'plateau = {Emax[i]:.0%}, WP = {(WP[i]/1000):.2f} kV, bkg gamma rate = {txt:.1f} kHz/cm^{{2}}, Eff(WP) = {fits[i].Eval(WP[i]):.0%}'
        legend.AddEntry(graphs[i], eff_i_STDMX, "p")

mg.GetYaxis().SetRangeUser(0, 1.4)
mg.GetXaxis().SetRangeUser(6300, 7600)
mg.Draw("AP")

line = TLine(6300, 1., 7600, 1.)
line.SetLineColor(1)
line.SetLineStyle(9)
line.SetLineWidth(2)
line.Draw()
legend.Draw()  

c1.Draw()
c1.SaveAs("eff_vs_HV_with_fits.pdf")